In [1]:
####################################################################################################
# Version 1 ########################################################################################
# Author: Alexander O. Smith
# Created: Oct 25, 2021
# Updated: Nov 1, 2021
####################################################################################################
# IMPORTS ##########################################################################################
from bs4 import BeautifulSoup as bs  #To Output Clean HTML files (and general scraping)
from lxml import html, etree         #For XPath
from io import StringIO, BytesIO
from urllib.request import Request, urlopen
import urllib.request
import pandas as pd
import numpy as np
from datetime import date
import os, re, traceback
####################################################################################################
def kym_gallery_pull(name='rickroll'):

    # Setup an output file...
    out = open(f'{name}_collection_output.txt',"a+")
    today = date.today()
    dateMDY = today.strftime("%m-%d-%y")
    date_time = today.strftime("%m/%d/%Y, %H:%M:%S")
    out.write(f'KYM_GALLERY_PULL Function Initiated: {date_time}...')
    print(f'KYM_GALLERY_PULL Function Initiated: {date_time}...')

    # Try/Except captures errors in the outfile: 'out'
    try:
        # Set up url connection with BeautifulSoup
        name = re.sub('\s', '-', str(name))
        out.write(f'Collecting image gallery "{name}" from KnowYourMeme.\n')
        #print(f'Collecting image gallery "{name}" from KnowYourMeme.')
        url = f'https://knowyourmeme.com/memes/{name}'
        memeURL = f'{url}/photos/page/1'
        req = Request(str(memeURL), headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = bs(webpage, 'html.parser', from_encoding="iso-8859-1")

        # Load webpage into ELement Tree format for XPath search for first page image count:
        root = etree.HTML(webpage)
        result = etree.tostring(root, pretty_print=True, method="html")

        n = 1 #begin iterable for each KYM Gallery Page URL
        url_suffixes = [] #being the list of url directorys after the url prefix for each image

        # While loop runs so long as the photo_gallery is not empty
        while len(root.xpath("//a[@rel='photo_gallery']")) != 0:

            # Set up iterable url connection
            out.write(f'Retrieving image hrefs from page {n} of {name} gallery...\n')
            #print(f'Retrieving image hrefs from page {n} of {name} gallery...')
            memeURL = f'{url}/photos/page/{n}'
            req = Request(memeURL, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()
            root = etree.HTML(webpage)
            result = etree.tostring(root, pretty_print=True, method="html")

            url_suffixes = url_suffixes + root.xpath(f'//div/div[@id="photo_gallery"]/div[@class="item"]/a/@href')

            # Iterate to the next n to get the next KYM Gallery Page in 'memeURL' above
            n += 1

        out.write(f'Completed gallery href collection for {name} gallery.\n')
        #print(f'Completed gallery href collection for {name} gallery.')

        # Debugging to check to make sure we got all the images in the gallery
        total_url = len(url_suffixes)
        unique_url = len(pd.unique(url_suffixes))
        if total_url != unique_url:
            out.write(f'''COLLECTION WARNING: Difference between the total URLs - unique URLs is non-zero.\n
                This indicates a loss of {total_url - unique_url} gallery images from KYM.\n
                The total number of unique URLs is {unique_url}.\n''')
            #print(f'''COLLECTION WARNING: Difference between the total URLs - unique URLs is non-zero.\n
            #      This indicates a loss of {total_url - unique_url} gallery images from KYM.\n
            #      The total number of unique URLs is {unique_url}.''')
        else:
            out.write(f'The total number of URLs is {unique_url}.\n')
            #print(f'The total number of URLs is {unique_url}.')

        # Building lists for a metadata dataframe
        lrg_img = []          # large img URLs
        img_IDs = []          # img_IDs
        img_locs = []         # locations of image
        alt_txt = []          # image alt text
        date_upload = []      # image upload date
        views = []            # number of views of each image
        views_today = []      # number of views today for each image
        img_source = []       # given source of image
        pos_votes = []        # positive votes on an image
        neg_votes = []        # negative votes on an image
        kym_tags = []         # tags offered by KYM
        img_edits_url = []    # url extension to image metadata edits
        author = []           # author handle assigned to image

        # Loop saves images and metadata for each image.
        for u in url_suffixes:
            out.write(f'Retrieving img at href {u}\n')
            print(f'Retrieving img at href {u}')
            imgURL = f'{url}{u}'
            out.write(imgURL+'\n')
            print(imgURL)
            reqIMG = Request(str(imgURL), headers={'User-Agent': 'Mozilla/5.0'})
            imgpage = urlopen(reqIMG).read()
            soup = bs(imgpage, 'html.parser') #, from_encoding="iso-8859-1"
            # Load webpage into ELement Tree format for XPath search for first page image count:
            imgRoot = etree.HTML(imgpage)
            # Putting all final image URL locations into 'lrg_img' list.
            img = imgRoot.xpath("*//div[@class='thumbable']//img[contains(@data-src, 'newsfeed')]//@data-src")[0]
            lrg_img.append(img)

            # Using 'img' urls to create a unique ID
            i = img.partition("newsfeed/")[2]
            img_file = re.sub('/', '_',i)
            out.write(f'Retrieving Image {img_file}\n')
            print(f'Retrieving Image {img_file}')
            img_ID = img_file.partition('.')[0]
            # Should be a unique image ID
            img_ID = f'IMG_{img_ID}'
            img_IDs.append(img_ID)

            # Creating an image location name, and making a metadata variable for it
            img_loc = f'{str(name)}-meme'
            img_locs.append(img_loc)

            # KYM image metadata imgRoot location, under "Image Details"
            # For explanation of meanings of the appended metadata, see empty lists defined before loop.
            view = imgRoot.xpath(
              "//aside[@id='sidebar']//span[@class='view_count']/text()")[0]
            views.append(view)
            view_today = imgRoot.xpath(
              "//aside[@id='sidebar']//span[@class='today_view_count']/text()")[0][1:-1] 
            view_today = re.sub('\sfrom\stoday', '', view_today)
            views_today.append(view_today)
            upload = imgRoot.xpath(
              "//aside[@id='sidebar']//abbr[@class='timeago']/@title")[0]
            date_upload.append(upload)
            try:
                source = imgRoot.xpath(
                  "//aside[@id='sidebar']//div//p/a[@href]/@href")[1]
            except IndexError:
                source = 'No Source Given'
            img_source.append(source)
            thumbs_alt = imgRoot.xpath(
              "//div[@class='thumbable']//div[@class='thumb_mini_container']//span/@alt")[0]
            thumbs_alt = thumbs_alt.split(', ')
            thumbs_alt[0] = re.sub('\sup', '',thumbs_alt[0])
            thumbs_alt[1] = re.sub('\sdown', '',thumbs_alt[1])
            pos_votes.append(thumbs_alt[0])
            neg_votes.append(thumbs_alt[1])
            tag = imgRoot.xpath(
              "//aside[@id='sidebar']//div//p[@id='tag_list']//@data-tag")
            delim = '|'
            tags = ''
            for t in tag:
                tags = tags + str(t) + delim 
            kym_tags.append(tags)
            edits =str(
              'https://knowyourmeme.com'
              +imgRoot.xpath("//aside[@id='sidebar']//div[@class='tc']//a[contains(@href, 'edits')]/@href")[0])
            img_edits_url.append(edits)
            alt =  imgRoot.xpath("*//div[@class='thumbable']//img[@alt]/@alt")[0]
            alt_txt.append(alt)

            author_tag = imgRoot.xpath("//div[@id='author_info']//div//h6/a/text()")
            author.append(author_tag)

            # Output Image Files To Created Meme Directory
            os.makedirs(f'./GCV_FuncTestData/{img_loc}', exist_ok=True)
            urllib.request.urlretrieve(img, f'./GCV_FuncTestData/{img_loc}/{img_file}')

            # Build dataframe of metadata
            # Jeff says "short columns sooner, long columns later... with the more important columns prioritized sooner."
        metadata_df = pd.DataFrame({
            'img_ID': img_IDs, 
            'meme_name': img_locs,
            'date_upload': date_upload,
            'kym_tags': kym_tags,
            'pos_votes': pos_votes,
            'neg_votes': neg_votes,
            'img_source': img_source,
            'views': views,
            'views_today': views_today,
            'img_edits_url': img_edits_url,
            'author': author,
            'alt_txt': alt_txt,
            'img_url': url_suffixes,
            'lrg_img_url': lrg_img
        })

        # Output a metadata csv with csv name containing the date captured.
        metadata_df.to_csv(
            f'./GCV_FuncTestData/{str(name)}-meme/{name}_KYMmetadata_{dateMDY}.csv'
          , index=False)
    except:
        traceback.print_exc(file=out)
    out.write('Metadata file completed.\n')
    #print('Metadata file completed.')
    out.write('KYM_GALLERY_PULL Function Completed.\n\n')
    #print('KYM_GALLERY_PULL Function Completed.\n\n')
    # Close Output File
    out.close()

In [2]:
kym_gallery_pull('hello there')

KYM_GALLERY_PULL Function Initiated: 12/29/2021, 00:00:00...
Retrieving img at href /photos/2146169-hello-there
https://knowyourmeme.com/memes/hello-there/photos/2146169-hello-there
Retrieving Image 002_146_169_595.jpg
Retrieving img at href /photos/2100692-hello-there
https://knowyourmeme.com/memes/hello-there/photos/2100692-hello-there
Retrieving Image 002_100_692_e5a.jpg
Retrieving img at href /photos/2087100-hello-there
https://knowyourmeme.com/memes/hello-there/photos/2087100-hello-there
Retrieving Image 002_087_100_63a.gif


In [1]:
# Metadata Test Script

def kym_md_test(name='rickroll'):
    
    # Set up url connection with BeautifulSoup
    name = re.sub('\s', '-', str(name))
    url = f'https://knowyourmeme.com/memes/{name}'
    memeURL = f'{url}/photos/page/1'
    req = Request(str(memeURL), headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = bs(webpage, 'html.parser', from_encoding="iso-8859-1")

    # Load webpage into ELement Tree format for XPath search for first page image count:
    root = etree.HTML(webpage)
    result = etree.tostring(root, pretty_print=True, method="html")
    
    url_suffixes = root.xpath(f'//div/div[@id="photo_gallery"]/div[@class="item"]/a/@href')
        
    # Building lists for a metadata dataframe
    lrg_img = []          # large img URLs
    img_IDs = []          # img_IDs
    img_locs = []         # locations of image
    alt_txt = []          # image alt text
    date_upload = []      # image upload date
    views = []            # number of views of each image
    views_today = []      # number of views today for each image
    img_source = []       # given source of image
    pos_votes = []        # positive votes on an image
    neg_votes = []        # negative votes on an image
    kym_tags = []         # tags offered by KYM
    img_edits_url = []    # url extension to image metadata edits
    author = []           # author handle assigned to image
    
    # Loop saves images and metadata for each image.
    for u in url_suffixes:
        imgURL = f'{url}{u}'
        reqIMG = Request(str(imgURL), headers={'User-Agent': 'Mozilla/5.0'})
        imgpage = urlopen(reqIMG).read()
        soup = bs(imgpage, 'html.parser') #, from_encoding="iso-8859-1"
        # Load webpage into ELement Tree format for XPath search for first page image count:
        imgRoot = etree.HTML(imgpage)
        # Putting all final image URL locations into 'lrg_img' list.
        img = imgRoot.xpath("*//div[@class='thumbable']//img[contains(@data-src, 'newsfeed')]//@data-src")[0]
        
        # Using 'img' urls to create a unique ID
        i = img.partition("newsfeed/")[2]
        img_file = re.sub('/', '_',i)
        img_ID = img_file.partition('.')[0]
        # Should be a unique image ID
        img_ID = f'IMG_{img_ID}'
        img_IDs.append(img_ID)
        
        # Creating an image location name, and making a metadata variable for it
        img_loc = f'{str(name)}-meme'
        img_locs.append(img_loc)  
        
        #thumb_up = imgRoot.xpath(
        #    "//div[@class='thumbable']//div[@class='thumb_mini_container']//text()")[0]
        #print(thumb_up)
        thumbs_alt = imgRoot.xpath(
            "//div[@class='thumbable']//div[@class='thumb_mini_container']//span/@alt")[0]
        thumbs_alt = thumbs_alt.split(', ')
        thumbs_alt[0] = re.sub('\sup', '',thumbs_alt[0])
        thumbs_alt[1] = re.sub('\sdown', '',thumbs_alt[1])
        print(thumbs_alt[1])
    

In [5]:
#kym_md_test()

In [36]:
url_pg = 'https://knowyourmeme.com/memes/all'
req_pg = Request(url_pg, headers={'User-Agent': 'Mozilla/5.0'})
webpage_pg = urlopen(req_pg).read()
soup = bs(webpage_pg, 'html.parser', from_encoding="iso-8859-1")

# Load webpage into ELement Tree format for XPath search for first page image count:
root_pg = etree.HTML(webpage_pg)
result = etree.tostring(root_pg, pretty_print=True, method="html")

n = 1 #begin iterable for each KYM Gallery Page URL
url_suffixes = [] #being the list of url directorys after the url prefix for each image

# Get the count of the number of meme entries expected to collect for outfile.
meme_count = root_pg.xpath("//header[@id='section_header']/hgroup/p/text()")[0]
meme_count = re.findall('[0-9,\,]+', meme_count)[0]
print(f'KnowYourMeme reports a count of {meme_count} meme entries.')
meme_count = re.sub('\,','',meme_count)

# Empty Lists for Metadata Output
meme_names = []
meme_sufs = []
meme_gallery = []
meme_labels = []
entry_id = []

# Initiate iterable for meme entry page
n = 1
# While loop runs so long as the photo_gallery is not empty
url_len = len(root_pg.xpath("//section[@id='entries']//table[@class='entry_list']//td[contains(@class, 'entry_')]//a[@href]//@href"))
while  url_len != 0:
    # Connecting to Meme Entry's Page N
    print(f'Retrieving meme hrefs from page {n} of All Memes on KYM...')
    req_pg = Request(url_pg, headers={'User-Agent': 'Mozilla/5.0'})
    root_pg = etree.HTML(webpg)
    result = etree.tostring(root_pg, pretty_print=True, method="html")
    xpth = "//section[@id='entries']//table[@class='entry_list']//td[contains(@class, 'entry_')]/a[@href]//@href"
    meme_sufs = meme_sufs + root_pg.xpath(xpth)
    xpth2 = "//section[@id='entries']//table[@class='entry_list']//td[contains(@class, 'entry_')]/a/img[@title]/@title"
    meme_names = meme_names + root_pg.xpath(xpth2)
    
    xpth2 = "//section[@id='entries']//table[@class='entry_list']//td[contains(@class, 'entry_')]/@class"
    entry = root_pg.xpath(xpth2)
    
    for id in entry:
        labs = f"//section[@id='entries']//table[@class='entry_list']//td[@class='{id}']//span/text()"
        lab = root_pg.xpath(labs)
        meme_labels.append(lab)
        #print(meme_labels)
        
    # Figure out how to distinquish labs for each meme entry.
    n+=1
    url_pg = f'{url}/page/{n}'
    url_len = len(root_pg.xpath(
        "//section[@id='entries']//table[@class='entry_list']//td[contains(@class, 'entry_')]"))

# From the list of meme_names create the url to each meme submission.
for i in range(1,len(meme_names)):
    meme_url = 'https://knowyourmeme.com/' + meme_sufs[i] + '/photos/'
    meme_gallery = meme_gallery + meme_url
    
    #labs = "//section[@id='entries']//table[@class='entry_list']//td[contains(@class, 'entry_')]//div[@class = entry_labels]//span/text()"
    #print(meme_sufs)
    # Iterate to the next n to get the next KYM Gallery Page in 'memeURL' above
        # Output a metadata csv with csv name containing the date captured.
metadata_df = pd.DataFrame({
    'meme_entry_ID': entry_id,
    'meme_names': meme_names,
    'meme_sufs' : meme_sufs,
    'meme_gallery' : meme_gallery,
    'meme_labels' : meme_labels
})
        
metadata_df.to_csv(
    f'./GCV_FuncTestData/{name}_KYMEntry_{dateMDY}.csv'
  , index=False)

print(f'Retreived all meme galleries.\n')

KnowYourMeme reports a count of 29,466 meme entries.
Retrieving meme hrefs from page 1 of All Memes on KYM...
Retrieving meme hrefs from page 2 of All Memes on KYM...
Retrieving meme hrefs from page 3 of All Memes on KYM...
Retrieving meme hrefs from page 4 of All Memes on KYM...
Retrieving meme hrefs from page 5 of All Memes on KYM...
Retrieving meme hrefs from page 6 of All Memes on KYM...
Retrieving meme hrefs from page 7 of All Memes on KYM...
Retrieving meme hrefs from page 8 of All Memes on KYM...
Retrieving meme hrefs from page 9 of All Memes on KYM...
Retrieving meme hrefs from page 10 of All Memes on KYM...
Retrieving meme hrefs from page 11 of All Memes on KYM...
Retrieving meme hrefs from page 12 of All Memes on KYM...
Retrieving meme hrefs from page 13 of All Memes on KYM...
Retrieving meme hrefs from page 14 of All Memes on KYM...
Retrieving meme hrefs from page 15 of All Memes on KYM...
Retrieving meme hrefs from page 16 of All Memes on KYM...
Retrieving meme hrefs from p

Retrieving meme hrefs from page 151 of All Memes on KYM...
Retrieving meme hrefs from page 152 of All Memes on KYM...
Retrieving meme hrefs from page 153 of All Memes on KYM...
Retrieving meme hrefs from page 154 of All Memes on KYM...
Retrieving meme hrefs from page 155 of All Memes on KYM...
Retrieving meme hrefs from page 156 of All Memes on KYM...
Retrieving meme hrefs from page 157 of All Memes on KYM...
Retrieving meme hrefs from page 158 of All Memes on KYM...
Retrieving meme hrefs from page 159 of All Memes on KYM...
Retrieving meme hrefs from page 160 of All Memes on KYM...
Retrieving meme hrefs from page 161 of All Memes on KYM...
Retrieving meme hrefs from page 162 of All Memes on KYM...
Retrieving meme hrefs from page 163 of All Memes on KYM...
Retrieving meme hrefs from page 164 of All Memes on KYM...
Retrieving meme hrefs from page 165 of All Memes on KYM...
Retrieving meme hrefs from page 166 of All Memes on KYM...
Retrieving meme hrefs from page 167 of All Memes on KYM.

Retrieving meme hrefs from page 292 of All Memes on KYM...
Retrieving meme hrefs from page 293 of All Memes on KYM...
Retrieving meme hrefs from page 294 of All Memes on KYM...
Retrieving meme hrefs from page 295 of All Memes on KYM...
Retrieving meme hrefs from page 296 of All Memes on KYM...
Retrieving meme hrefs from page 297 of All Memes on KYM...
Retrieving meme hrefs from page 298 of All Memes on KYM...
Retrieving meme hrefs from page 299 of All Memes on KYM...
Retrieving meme hrefs from page 300 of All Memes on KYM...
Retrieving meme hrefs from page 301 of All Memes on KYM...
Retrieving meme hrefs from page 302 of All Memes on KYM...
Retrieving meme hrefs from page 303 of All Memes on KYM...
Retrieving meme hrefs from page 304 of All Memes on KYM...
Retrieving meme hrefs from page 305 of All Memes on KYM...
Retrieving meme hrefs from page 306 of All Memes on KYM...
Retrieving meme hrefs from page 307 of All Memes on KYM...
Retrieving meme hrefs from page 308 of All Memes on KYM.

Retrieving meme hrefs from page 439 of All Memes on KYM...
Retrieving meme hrefs from page 440 of All Memes on KYM...
Retrieving meme hrefs from page 441 of All Memes on KYM...
Retrieving meme hrefs from page 442 of All Memes on KYM...
Retrieving meme hrefs from page 443 of All Memes on KYM...
Retrieving meme hrefs from page 444 of All Memes on KYM...
Retrieving meme hrefs from page 445 of All Memes on KYM...
Retrieving meme hrefs from page 446 of All Memes on KYM...
Retrieving meme hrefs from page 447 of All Memes on KYM...
Retrieving meme hrefs from page 448 of All Memes on KYM...
Retrieving meme hrefs from page 449 of All Memes on KYM...
Retrieving meme hrefs from page 450 of All Memes on KYM...
Retrieving meme hrefs from page 451 of All Memes on KYM...
Retrieving meme hrefs from page 452 of All Memes on KYM...
Retrieving meme hrefs from page 453 of All Memes on KYM...
Retrieving meme hrefs from page 454 of All Memes on KYM...
Retrieving meme hrefs from page 455 of All Memes on KYM.

Retrieving meme hrefs from page 588 of All Memes on KYM...
Retrieving meme hrefs from page 589 of All Memes on KYM...
Retrieving meme hrefs from page 590 of All Memes on KYM...
Retrieving meme hrefs from page 591 of All Memes on KYM...
Retrieving meme hrefs from page 592 of All Memes on KYM...
Retrieving meme hrefs from page 593 of All Memes on KYM...
Retrieving meme hrefs from page 594 of All Memes on KYM...
Retrieving meme hrefs from page 595 of All Memes on KYM...
Retrieving meme hrefs from page 596 of All Memes on KYM...
Retrieving meme hrefs from page 597 of All Memes on KYM...
Retrieving meme hrefs from page 598 of All Memes on KYM...
Retrieving meme hrefs from page 599 of All Memes on KYM...
Retrieving meme hrefs from page 600 of All Memes on KYM...
Retrieving meme hrefs from page 601 of All Memes on KYM...
Retrieving meme hrefs from page 602 of All Memes on KYM...
Retrieving meme hrefs from page 603 of All Memes on KYM...
Retrieving meme hrefs from page 604 of All Memes on KYM.

...

Retrieving meme hrefs from page 42368 of All Memes on KYM...
Retrieving meme hrefs from page 42369 of All Memes on KYM...
Retrieving meme hrefs from page 42370 of All Memes on KYM...
Retrieving meme hrefs from page 42371 of All Memes on KYM...
Retrieving meme hrefs from page 42372 of All Memes on KYM...
Retrieving meme hrefs from page 42373 of All Memes on KYM...
Retrieving meme hrefs from page 42374 of All Memes on KYM...
Retrieving meme hrefs from page 42375 of All Memes on KYM...
Retrieving meme hrefs from page 42376 of All Memes on KYM...
Retrieving meme hrefs from page 42377 of All Memes on KYM...
Retrieving meme hrefs from page 42378 of All Memes on KYM...
Retrieving meme hrefs from page 42379 of All Memes on KYM...
Retrieving meme hrefs from page 42380 of All Memes on KYM...
Retrieving meme hrefs from page 42381 of All Memes on KYM...
Retrieving meme hrefs from page 42382 of All Memes on KYM...
Retrieving meme hrefs from page 42383 of All Memes on KYM...
Retrieving meme hrefs fr

Retrieving meme hrefs from page 42506 of All Memes on KYM...
Retrieving meme hrefs from page 42507 of All Memes on KYM...
Retrieving meme hrefs from page 42508 of All Memes on KYM...
Retrieving meme hrefs from page 42509 of All Memes on KYM...
Retrieving meme hrefs from page 42510 of All Memes on KYM...
Retrieving meme hrefs from page 42511 of All Memes on KYM...
Retrieving meme hrefs from page 42512 of All Memes on KYM...
Retrieving meme hrefs from page 42513 of All Memes on KYM...
Retrieving meme hrefs from page 42514 of All Memes on KYM...
Retrieving meme hrefs from page 42515 of All Memes on KYM...
Retrieving meme hrefs from page 42516 of All Memes on KYM...
Retrieving meme hrefs from page 42517 of All Memes on KYM...
Retrieving meme hrefs from page 42518 of All Memes on KYM...
Retrieving meme hrefs from page 42519 of All Memes on KYM...
Retrieving meme hrefs from page 42520 of All Memes on KYM...
Retrieving meme hrefs from page 42521 of All Memes on KYM...
Retrieving meme hrefs fr

Retrieving meme hrefs from page 42643 of All Memes on KYM...
Retrieving meme hrefs from page 42644 of All Memes on KYM...
Retrieving meme hrefs from page 42645 of All Memes on KYM...
Retrieving meme hrefs from page 42646 of All Memes on KYM...
Retrieving meme hrefs from page 42647 of All Memes on KYM...
Retrieving meme hrefs from page 42648 of All Memes on KYM...
Retrieving meme hrefs from page 42649 of All Memes on KYM...
Retrieving meme hrefs from page 42650 of All Memes on KYM...
Retrieving meme hrefs from page 42651 of All Memes on KYM...
Retrieving meme hrefs from page 42652 of All Memes on KYM...
Retrieving meme hrefs from page 42653 of All Memes on KYM...
Retrieving meme hrefs from page 42654 of All Memes on KYM...
Retrieving meme hrefs from page 42655 of All Memes on KYM...
Retrieving meme hrefs from page 42656 of All Memes on KYM...
Retrieving meme hrefs from page 42657 of All Memes on KYM...
Retrieving meme hrefs from page 42658 of All Memes on KYM...
Retrieving meme hrefs fr

Retrieving meme hrefs from page 42779 of All Memes on KYM...
Retrieving meme hrefs from page 42780 of All Memes on KYM...
Retrieving meme hrefs from page 42781 of All Memes on KYM...
Retrieving meme hrefs from page 42782 of All Memes on KYM...
Retrieving meme hrefs from page 42783 of All Memes on KYM...
Retrieving meme hrefs from page 42784 of All Memes on KYM...
Retrieving meme hrefs from page 42785 of All Memes on KYM...
Retrieving meme hrefs from page 42786 of All Memes on KYM...
Retrieving meme hrefs from page 42787 of All Memes on KYM...
Retrieving meme hrefs from page 42788 of All Memes on KYM...
Retrieving meme hrefs from page 42789 of All Memes on KYM...
Retrieving meme hrefs from page 42790 of All Memes on KYM...
Retrieving meme hrefs from page 42791 of All Memes on KYM...
Retrieving meme hrefs from page 42792 of All Memes on KYM...
Retrieving meme hrefs from page 42793 of All Memes on KYM...
Retrieving meme hrefs from page 42794 of All Memes on KYM...
Retrieving meme hrefs fr

KeyboardInterrupt: 

In [28]:
meme_sufs

['/memes/thrembo',
 '/memes/subcultures/murder-drones',
 '/memes/sardaukar-chant',
 '/memes/events/niaids-funding-of-dog-experiments',
 '/memes/colonel-toad',
 '/memes/people/the-poopshitters--2',
 '/memes/stickman-irl-tiktok-trend',
 '/memes/why-are-you-here-i-thought-you-were-financially-stable',
 '/memes/subcultures/come-and-learn-with-pibby',
 '/memes/optimum-pride',
 '/memes/subcultures/meta',
 '/memes/the-dumbest-thing-ive-ever-heard',
 '/memes/the-10000-years-hiatus-ichikawas-ps5',
 '/memes/akira-movie-poster-parodies',
 '/memes/events/october-2021-roblox-outage',
 '/memes/events/teamseas',
 '/memes/walter-white-cooking',
 '/memes/netflix-are-you-still-watching-someones-daughter',
 '/memes/ah-gottem-ggs',
 '/memes/gibby-requires-soil-coochie',
 '/memes/people/tweak-season',
 '/memes/charlie-brown-football-gag',
 '/memes/karin-asaka-works-at-kfc',
 '/memes/mom-dad-barts-dead',
 '/memes/bathroom-shrines',
 '/memes/how-did-you-do-in-pe-today',
 '/memes/facebook-name-change-to-meta'